In [ ]:
import requests, json
from datetime import datetime
slug = "maduro-in-us-custody-by-january-31"

url = f"https://gamma-api.polymarket.com/markets/slug/{slug}"

response = requests.get(url)

print(json.dumps(json.loads(response.text), indent=4))

{
    "id": "1094682",
    "question": "Maduro in U.S. custody by January 31?",
    "conditionId": "0xbfa45527ec959aacc36f7c312bd4f328171a7681ef1aeb3a7e34db5fb47d3f1d",
    "slug": "maduro-in-us-custody-by-january-31",
    "resolutionSource": "",
    "endDate": "2026-01-31T00:00:00Z",
    "startDate": "2026-01-03T07:49:39.785553Z",
    "image": "https://polymarket-upload.s3.us-east-2.amazonaws.com/maduro-in-us-custody-by-january-31-ljXbIe3q5Y2o.jpg",
    "icon": "https://polymarket-upload.s3.us-east-2.amazonaws.com/maduro-in-us-custody-by-january-31-ljXbIe3q5Y2o.jpg",
    "description": "This market will resolve to \u201cYes\u201d if Venezuelan President Nicolas Maduro is taken into custody by the U.S. government at any point by January 31, 2026, 11:59 PM ET. Otherwise, this market will resolve to \u201cNo\u201d.\n\nMaduro will be considered taken into U.S. government custody if U.S. government personnel (including military, CIA, personnel of another agency, or contractors acting under

In [2]:
clob_token_id= "7023463941941580393623777508894165086142929841541805476418845616988817847686"
start_time = datetime.datetime(2026,1,1,0,0).timestamp()
end_time = datetime.datetime(2026,1,3,0,0).timestamp()

print(start_time)
print(end_time)


url = f"https://clob.polymarket.com/prices-history?market={clob_token_id}&startTs={start_time}&endTs={end_time}&fidelity=&3600interval=max"

response = requests.get(url)
print(json.dumps(json.loads(response.text), indent=4))

1767254400.0
1767427200.0
{
    "history": [
        {
            "t": 1767427042,
            "p": 0.5
        },
        {
            "t": 1767427091,
            "p": 0.5
        },
        {
            "t": 1767427151,
            "p": 0.285
        }
    ]
}


In [12]:
#this gets all the markets
market_url = "https://gamma-api.polymarket.com/markets"

response = requests.get(market_url)

#print(json.dumps(json.loads(response.text), indent=4))

In [ ]:
#some interesting slugs to investigate: 
vaccine_2020slug = "will-there-be-an-emergency-use-authorization-granted-for-a-covid-19-vaccine-prior-to-nov-3"
drake_2020slug = "will-drake-release-a-new-studio-album-before-the-end-of-2020"
vaccine_url = f"https://gamma-api.polymarket.com/markets/slug/{vaccine_2020slug}"
drake_url = f"https://gamma-api.polymarket.com/markets/slug/{drake_2020slug}"

In [52]:
response_vaccine_raw = requests.get(vaccine_url)
response_vaccine = response_vaccine_raw.json()
vaccine_clob_ids = response_vaccine['clobTokenIds']
vaccine_created = response_vaccine['createdAt']
vaccine_closed = response_vaccine['closedTime']
vaccine_volume = response_vaccine['volume']

In [57]:
vaccine_closed

'2020-11-02 16:33:23+00'

In [43]:
response_drake_raw = requests.get(drake_url)
response_drake = response_drake_raw.json()
drake_clob_ids = response_drake['clobTokenIds']
drake_created = response_drake['createdAt']
drake_closed = response_drake['closedTime']
drake_voliume = response_drake['volume']

In [58]:

vaccine_created_utc = vaccine_created[:-1] + '+00:00'
vaccine_created_dt = datetime.fromisoformat(vaccine_created_utc)
vaccine_closed_utc = vaccine_closed + ':00'
vaccine_closed_dt = datetime.fromisoformat(vaccine_closed_utc)

In [59]:
vaccine_history_url = f"https://clob.polymarket.com/prices-history?market={vaccine_clob_ids[0]}&startTs={vaccine_created_dt}&endTs={vaccine_closed_dt}&fidelity=&3600interval=max"
vaccine_history_response = requests.get(vaccine_history_url)
print(json.dumps(json.loads(vaccine_history_response.text), indent=4))

{
    "error": "strconv.ParseFloat: parsing \"2020-10-12 22:00:05.783000 00:00\": invalid syntax"
}
